In [1]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
import math
import numpy as np
from IPython.display import display

In [2]:
#para saber el punto inicial y final de la linea
line_438_busstops = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Paradas/gtfs_linha438-paradas.csv')

departure_points = line_438_busstops[(line_438_busstops["sequencia"]==1)]
departure_points


,linha,descricao,agencia,sequencia,latitude,longitude
94,438,438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR,Fetranspor,1,-22.9882,-43.2283
127,438,438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR,Fetranspor,1,-22.9163,-43.2512


In [3]:
#Estableciendo la conexion con la base de datos aniv_cidade_event en MongoDB
db = MongoClient().aniv_cidade_event

In [4]:
#Funcion para determinar el punto inicial y final de un recorrido de una linea de omnibus
def bus_line_initial_final_points(bus_line):
    busstops_firstpoints=[]
    for row in db.busstops.find({ "linha": bus_line,"sequencia": 1 }):
        busstops_firstpoints.insert(0, [row[u'linha'], row[u'descricao'], row[u'latitude'], row[u'longitude']])

    #convertir la lista en un dataframe de pandas
    df_departure_points = pd.DataFrame(busstops_firstpoints, columns=['linha','descricao', 'latitude', 'longitude'])
    #df_departure_points
    initial_point = df_departure_points['latitude'][0], df_departure_points['longitude'][0]
    inverted_initial_point = df_departure_points['longitude'][0], df_departure_points['latitude'][0]
    final_point = df_departure_points['latitude'][1], df_departure_points['longitude'][1]
    inverted_final_point = df_departure_points['longitude'][1], df_departure_points['latitude'][1]
    return initial_point, final_point, inverted_initial_point, inverted_final_point

In [5]:
#Llamada a la funcion bus_line_initial_final_points de la 438
initial_final_points = bus_line_initial_final_points(438)
initial_final_points

((-22.9163, -43.251199999999997),
 (-22.988199999999999, -43.228299999999997),
 (-43.251199999999997, -22.9163),
 (-43.228299999999997, -22.988199999999999))

In [6]:
#De la misma llamada a la funcion de arriba retornar el valor en la posicion 1
initial_point = initial_final_points[1]
initial_point

(-22.988199999999999, -43.228299999999997)

In [7]:
#De la misma llamada a la funcion de arriba retornar el valor en la posicion 0
final_point = initial_final_points[0]
final_point

(-22.9163, -43.251199999999997)

In [8]:
inverted_initial_point = initial_final_points[3]
inverted_initial_point

(-43.228299999999997, -22.988199999999999)

In [9]:
inverted_final_point = initial_final_points[2]
inverted_final_point

(-43.251199999999997, -22.9163)

In [10]:
#Function to compute the bus_ids circulating during a day 20150208 of bus line 438
def bus_id_list ():
    bus_id_20150208_438 = db.dados20150208_line_438.distinct("bus_id")
    return bus_id_20150208_438
    

In [11]:
#llamada a la funcion bus_id_list
busId_list = bus_id_list()
busId_list

[u'A27611',
 u'A27629',
 u'A27565',
 u'A27604',
 u'A27655',
 u'A27675',
 u'A27504',
 u'A27610',
 u'A27592',
 u'A27574',
 u'A27599',
 u'A27597',
 u'A27626',
 u'A27554',
 u'A27587',
 u'A27633',
 u'A27512',
 u'A27562',
 u'A27603',
 u'A27544',
 u'A27525',
 u'A27557',
 u'A27514',
 u'A27640',
 u'A27510',
 u'A27577',
 u'A27511']

In [12]:
#Funcion para meter los bus_Id del dataframe para un array
def get_busId_list():
    array_busId = []
    for bus in busId_list:
        strbus = str(bus)
        array_busId.insert(len(array_busId), strbus)
    return array_busId       
    

In [13]:
array_busId = get_busId_list()
array_busId

['A27611',
 'A27629',
 'A27565',
 'A27604',
 'A27655',
 'A27675',
 'A27504',
 'A27610',
 'A27592',
 'A27574',
 'A27599',
 'A27597',
 'A27626',
 'A27554',
 'A27587',
 'A27633',
 'A27512',
 'A27562',
 'A27603',
 'A27544',
 'A27525',
 'A27557',
 'A27514',
 'A27640',
 'A27510',
 'A27577',
 'A27511']

In [35]:
len(array_busId)

27

In [14]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150208_line_438.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.00052543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points


In [15]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos al 
#punto initial_point(-22.988199999999999, -43.228299999999997) aqui se pasan a la inversa
sorted_near_initial_points = near_points_to_origin ("A27514", (-43.228299999999997, -22.988199999999999))
sorted_near_initial_points


,timestamp,latitude,longitude,speed
0,08-02-2015 13:38:35,-22.988251,-43.228298,17
1,08-02-2015 16:44:35,-22.988350,-43.228260,21
2,08-02-2015 16:45:35,-22.988461,-43.228168,0
3,08-02-2015 16:46:05,-22.988480,-43.228149,0
4,08-02-2015 19:48:05,-22.988239,-43.228279,0


In [16]:
#LLamada a la funcion near_points_to_origin para encontrar los puntos más cercanos al 
#punto final_point (-22.9163, -43.251199999999997). Recordar que aqui las coordenadas del punto se pasan a la inversa(long, lat)
sorted_near_final_points = near_points_to_origin ("A27514", (-43.251199999999997, -22.9163))
sorted_near_final_points

,timestamp,latitude,longitude,speed
0,08-02-2015 15:11:35,-22.916161,-43.250881,0
1,08-02-2015 15:12:35,-22.916170,-43.250881,0
2,08-02-2015 15:13:35,-22.916161,-43.250881,0
3,08-02-2015 15:14:05,-22.916161,-43.250881,0
4,08-02-2015 15:15:35,-22.916161,-43.250881,0
5,08-02-2015 15:16:35,-22.916170,-43.250881,0
6,08-02-2015 15:17:35,-22.916161,-43.250881,0
7,08-02-2015 15:18:35,-22.916161,-43.250881,0
8,08-02-2015 15:19:35,-22.916161,-43.250881,0
9,08-02-2015 15:20:35,-22.916170,-43.250889,0


In [17]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [18]:
#pintando los puntos más cerca del punto inicial
fmap=folium.Map(location=[-22.950635, -43.210436], zoom_start=12)

for row in sorted_near_initial_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='violet', num_sides=4, radius=6)

embed_map(fmap)

In [53]:
#pintando los puntos más cerca del punto final (Vila Isabel)
for row in sorted_near_final_points.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='red', num_sides=4, radius=6)

embed_map(fmap)

In [20]:
#Pintando Circulos de radio de 1km alrededor de los puntos incial y final 
fmap.simple_marker(final_point, popup='Parada de Vila Isabel')
fmap.simple_marker(initial_point, popup='Parada Leblon')

#Punto inicial
fmap.circle_marker(location=final_point, radius=1000,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap.circle_marker(location=initial_point, radius=1000,line_color='blue',
                 fill_color='#3186cc', fill_opacity=0.2)

embed_map(fmap)

In [21]:
#Array de timestamp puntos near punto incial (Leblon)
initial_points_timestamp = sorted_near_initial_points['timestamp']
initial_points_timestamp


0    08-02-2015 13:38:35
1    08-02-2015 16:44:35
2    08-02-2015 16:45:35
3    08-02-2015 16:46:05
4    08-02-2015 19:48:05
Name: timestamp, dtype: object

In [22]:
final_points_timestamp = sorted_near_final_points['timestamp']
final_points_timestamp

0     08-02-2015 15:11:35
1     08-02-2015 15:12:35
2     08-02-2015 15:13:35
3     08-02-2015 15:14:05
4     08-02-2015 15:15:35
5     08-02-2015 15:16:35
6     08-02-2015 15:17:35
7     08-02-2015 15:18:35
8     08-02-2015 15:19:35
9     08-02-2015 15:20:35
10    08-02-2015 15:21:35
11    08-02-2015 15:22:35
12    08-02-2015 15:23:35
13    08-02-2015 15:24:35
14    08-02-2015 15:26:05
15    08-02-2015 15:26:35
16    08-02-2015 15:27:35
17    08-02-2015 15:28:35
18    08-02-2015 15:29:35
19    08-02-2015 15:30:35
20    08-02-2015 15:31:35
21    08-02-2015 15:32:35
22    08-02-2015 15:33:35
23    08-02-2015 15:34:35
24    08-02-2015 15:35:35
25    08-02-2015 15:36:35
26    08-02-2015 15:37:35
27    08-02-2015 15:38:35
28    08-02-2015 15:39:35
29    08-02-2015 15:40:35
             ...         
57    08-02-2015 18:26:05
58    08-02-2015 18:27:05
59    08-02-2015 18:27:35
60    08-02-2015 18:28:35
61    08-02-2015 18:30:05
62    08-02-2015 18:30:35
63    08-02-2015 18:31:05
64    08-02-

In [23]:
#Funcion que retorna el arreglo con las horas de salida del omnibus al punto inicial/final del trayecto
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif len(initial_points_timestamp_list) == 1:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours = []
    return list_departure_hours


In [24]:
#Llamada a la funcion bus_departure_hour para el punto Leblon (devuelve todas las horas en que salio el omnibus de Leblon)
leblon_departure_hours = bus_departure_hour(initial_points_timestamp)
leblon_departure_hours

[datetime.datetime(2015, 2, 8, 13, 38, 35),
 datetime.datetime(2015, 2, 8, 16, 46, 5),
 datetime.datetime(2015, 2, 8, 19, 48, 5)]

In [25]:
#Llamada a la funcion bus_departure_hour para el punto Vila Isabel  (devuelve todas las horas en que salio el omnibus de Vila Isabel)
vilaIsabel_departure_hours = bus_departure_hour(final_points_timestamp)
vilaIsabel_departure_hours

[datetime.datetime(2015, 2, 8, 15, 41, 35),
 datetime.datetime(2015, 2, 8, 18, 44, 35),
 datetime.datetime(2015, 2, 8, 20, 52, 35),
 datetime.datetime(2015, 2, 8, 22, 20, 43)]

In [26]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    if len(initial_points_timestamp_list) >1:
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif (len(initial_points_timestamp_list) == 1):
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_arrive_hours = []        
    return list_arrive_hours


In [27]:
#Llamada a la funcion bus_arrive_hour para el punto Leblon  (devuelve todas las horas en que llegó el omnibus a Leblon)
arrive_hour_to_leblon = bus_arrive_hour(initial_points_timestamp)
arrive_hour_to_leblon

[datetime.datetime(2015, 2, 8, 13, 38, 35),
 datetime.datetime(2015, 2, 8, 16, 44, 35),
 datetime.datetime(2015, 2, 8, 19, 48, 5)]

In [28]:
#Llamada a la funcion bus_arrive_hour para el punto Vila Isabel  (devuelve todas las horas en que llegó el omnibus a Vila Isabel)
arrive_hour_to_vilaisabel = bus_arrive_hour(final_points_timestamp)
arrive_hour_to_vilaisabel

[datetime.datetime(2015, 2, 8, 15, 11, 35),
 datetime.datetime(2015, 2, 8, 17, 51, 35),
 datetime.datetime(2015, 2, 8, 20, 52, 35),
 datetime.datetime(2015, 2, 8, 22, 12, 43)]

In [29]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [30]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            if(i+1 == len(departure_array_A)):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
         
            elif ((i+1 < len(departure_array_A)) and (departure_array_B[j] < departure_array_A[i+1])):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else: 
                i = i +1
            
        elif(departure_array_B[j] < departure_array_A[i]):
            if (j+1 == len(departure_array_B)):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
            elif ((j+1 < len(departure_array_B)) and (departure_array_A[i] < departure_array_B[j+1])):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else:
                j = j +1                
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  



In [ ]:
"""
 'A27611', 'A27629', 'A27565', 'A27604','A27655','A27675','A27504','A27610','A27592','A27574','A27599','A27597',
 'A27626','A27554','A27587','A27633','A27512','A27562','A27603','A27544','A27525','A27557','A27514','A27640','A27510',
 'A27577','A27511'
 """

In [40]:
#Funcion que mete todas las observaciones de GPS de un determinado bus_id en un vector
def build_vector_timestamp_latidude_longitude(bus_id): 
    vector = []
    for row in db.dados20150208_line_438.find({"bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        vector.insert(len(vector), [row[u'timestamp'], row[u'latitude'], row[u'longitude']])
    #convert the array in panda dataframe 
    result_vector = pd.DataFrame(vector, columns=['timestamp', 'latitude', 'longitude'])
    return result_vector

In [46]:
vector_A27611 = build_vector_timestamp_latidude_longitude('A27611')
#convertir el dataframe a lista
list_A27611 = vector_A27611.values.tolist()
list_A27611

[[u'08-02-2015 04:32:22', -22.916981, -43.210178000000006],
 [u'08-02-2015 04:31:52', -22.91667, -43.210121],
 [u'08-02-2015 04:30:52', -22.91132, -43.210331],
 [u'08-02-2015 00:03:23', -22.923901, -43.209831],
 [u'08-02-2015 00:02:53', -22.921110000000002, -43.209969],
 [u'08-02-2015 00:01:53', -22.916849, -43.210159000000004],
 [u'08-02-2015 00:00:23', -22.911261, -43.213428]]

In [44]:
#Funcion para calcular la distancia entre 2 puntos, teniendo en cuenta el radio de la tierra
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return (c * r)*1000 # return the distance in m

In [45]:
#funcion que dado un vector de lat, long y un punto A(lat,long), me devuelve un vector con la distancia de cada par al punto A

def vector_distancia(vector, point):
    vector_distance = []
    lat2 = point[0]
    long2 = point[1]
    for item in vector:
        lat1 = item[1]
        long1 = item[2]
        distance = haversine(long2, lat2, long1, lat1)
        latlon = (item[1], item[2])
        vector_distance.insert(len(vector_distance), [item[0],latlon, distance])
    df_vector_distance = pd.DataFrame(vector_distance, columns=['timestamp', 'latlon', 'distance'])
    return df_vector_distance   

In [56]:
dist_to_ini_A27611 = vector_distancia(list_A27611, initial_point)
sorted_dist_to_ini_A27611 = dist_to_ini_A27611.sort(["timestamp"])


dist_to_fin_A27611 = vector_distancia(list_A27611, final_point)
sorted_dist_to_fin_A27611 = dist_to_fin_A27611.sort(["timestamp"])
sorted_dist_to_ini_A27611

,timestamp,latlon,distance
6,08-02-2015 00:00:23,"(-22.911261, -43.213428)",8689.695597
5,08-02-2015 00:01:53,"(-22.916849, -43.210159)",8148.406173
4,08-02-2015 00:02:53,"(-22.92111, -43.209969)",7692.554286
3,08-02-2015 00:03:23,"(-22.923901, -43.209831)",7395.571787
2,08-02-2015 04:30:52,"(-22.91132, -43.210331)",8744.424230
1,08-02-2015 04:31:52,"(-22.91667, -43.210121)",8168.673286
0,08-02-2015 04:32:22,"(-22.916981, -43.210178)",8133.671336


In [52]:
#hacer una grafica interactiva con la libreria de python (plotly.plotly). Se instala por consola asi: sudo pip install plotly
#distancia en cada horario del dia del punto inicial (Leblon)
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd

#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_orange_stock.csv')
#df.head()

trace = Scatter( x=sorted_dist_to_ini_A27611['timestamp'], y = sorted_dist_to_ini_A27611['distance'] )
data = Data([trace])

# IPython notebook
# py.iplot(data, filename='pandas-time-series')

py.iplot(data, filename='pandas-time-series')

In [59]:
#pintando los puntos más cerca del punto inicial
fmap_A27611=folium.Map(location=[-22.950635, -43.210436], zoom_start=12)

for row in vector_A27611.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap_A27611.polygon_marker( latlon, fill_color='violet', num_sides=4, radius=6)

embed_map(fmap_A27611)

In [36]:
#dados para llamar a la funcion

mytabla1 = create_travel_table("Leblon", arrive_hour_to_leblon, leblon_departure_hours, "Vila Isabel", arrive_hour_to_vilaisabel, vilaIsabel_departure_hours, 'A27611', '438' )
mytabla1

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,438,A27611,Leblon_to_Vila Isabel,2015-02-08 13:38:35,2015-02-08 15:11:35,1:33
1,438,A27611,Vila Isabel_to_Leblon,2015-02-08 15:41:35,2015-02-08 16:44:35,1:03
2,438,A27611,Leblon_to_Vila Isabel,2015-02-08 16:46:05,2015-02-08 17:51:35,1:05
3,438,A27611,Vila Isabel_to_Leblon,2015-02-08 18:44:35,2015-02-08 19:48:05,1:03
4,438,A27611,Leblon_to_Vila Isabel,2015-02-08 19:48:05,2015-02-08 20:52:35,1:04


In [37]:
mytabla1_sorted = mytabla1.sort(["Departure_Hour"])
mytabla1_sorted

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,438,A27611,Leblon_to_Vila Isabel,2015-02-08 13:38:35,2015-02-08 15:11:35,1:33
1,438,A27611,Vila Isabel_to_Leblon,2015-02-08 15:41:35,2015-02-08 16:44:35,1:03
2,438,A27611,Leblon_to_Vila Isabel,2015-02-08 16:46:05,2015-02-08 17:51:35,1:05
3,438,A27611,Vila Isabel_to_Leblon,2015-02-08 18:44:35,2015-02-08 19:48:05,1:03
4,438,A27611,Leblon_to_Vila Isabel,2015-02-08 19:48:05,2015-02-08 20:52:35,1:04


In [32]:
#Funcion que crea la tabla de tiempo de viaje para cada bus_id de los que transitaron el el dia
def all_travel_table (line, punto_ini_coord_invertidas, name_punto_inicio, punto_fin_coord_invertidas, name_punto_final, array_busId):
    new_df = pd.DataFrame(columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])  
    for busId in array_busId:
        sorted_near_initial_points1 = near_points_to_origin(busId, punto_ini_coord_invertidas)
        sorted_near_final_points1 = near_points_to_origin(busId, punto_fin_coord_invertidas)
        initial_points_timestamp1 = sorted_near_initial_points1 ['timestamp']
        final_points_timestamp1 = sorted_near_final_points1 ['timestamp']
        #chequear si la lista initial_points_timestamp es diferente de vacio
        if len(initial_points_timestamp1) and len(final_points_timestamp1):
            vilaIsabel_departure_hours1 = bus_departure_hour(initial_points_timestamp1)
            first_point_departure_hour = bus_departure_hour (initial_points_timestamp1)
            last_point_departure_hour = bus_departure_hour (final_points_timestamp1)
            first_point_arrive_hour = bus_arrive_hour(initial_points_timestamp1)
            last_point_arrive_hour = bus_arrive_hour(final_points_timestamp1)
            my_time_table = create_travel_table(name_punto_inicio, first_point_arrive_hour, first_point_departure_hour, name_punto_final, last_point_arrive_hour, last_point_departure_hour, busId, line)
            new_df = new_df.append(my_time_table, ignore_index=True)
    return new_df


#el problema está en sorted_near_initial_points cuando devuelve vacio para algunos bus_id

In [33]:
query = all_travel_table ('438', inverted_initial_point, 'Leblon', inverted_final_point, 'Vila Isabel', array_busId)
query

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,438,A27629,Vila Isabel_to_Leblon,2015-02-08 18:23:57,2015-02-08 19:23:27,0:59
1,438,A27629,Leblon_to_Vila Isabel,2015-02-08 19:23:27,2015-02-08 20:28:57,1:05
2,438,A27565,Vila Isabel_to_Leblon,2015-02-08 05:45:40,2015-02-08 06:43:40,0:58
3,438,A27565,Leblon_to_Vila Isabel,2015-02-08 06:44:40,2015-02-08 07:41:40,0:57
4,438,A27565,Vila Isabel_to_Leblon,2015-02-08 07:58:40,2015-02-08 08:59:40,1:01
5,438,A27604,Vila Isabel_to_Leblon,2015-02-08 05:51:20,2015-02-08 14:20:03,8:28
6,438,A27604,Leblon_to_Vila Isabel,2015-02-08 14:20:03,2015-02-08 15:31:33,1:11
7,438,A27604,Vila Isabel_to_Leblon,2015-02-08 16:22:44,2015-02-08 17:17:14,0:54
8,438,A27604,Leblon_to_Vila Isabel,2015-02-08 17:17:14,2015-02-08 18:27:14,1:10
9,438,A27504,Vila Isabel_to_Leblon,2015-02-08 05:59:51,2015-02-08 06:51:21,0:51


In [34]:
sorted_query= query.sort(["Departure_Hour"])
sorted_query

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
2,438,A27565,Vila Isabel_to_Leblon,2015-02-08 05:45:40,2015-02-08 06:43:40,0:58
5,438,A27604,Vila Isabel_to_Leblon,2015-02-08 05:51:20,2015-02-08 14:20:03,8:28
9,438,A27504,Vila Isabel_to_Leblon,2015-02-08 05:59:51,2015-02-08 06:51:21,0:51
19,438,A27574,Vila Isabel_to_Leblon,2015-02-08 06:15:29,2015-02-08 17:28:54,11:13
21,438,A27554,Vila Isabel_to_Leblon,2015-02-08 06:15:51,2015-02-08 07:08:51,0:53
3,438,A27565,Leblon_to_Vila Isabel,2015-02-08 06:44:40,2015-02-08 07:41:40,0:57
23,438,A27562,Vila Isabel_to_Leblon,2015-02-08 06:45:28,2015-02-08 07:41:29,0:56
10,438,A27504,Leblon_to_Vila Isabel,2015-02-08 06:52:21,2015-02-08 07:41:51,0:49
22,438,A27554,Leblon_to_Vila Isabel,2015-02-08 07:08:51,2015-02-08 08:07:51,0:59
17,438,A27610,Vila Isabel_to_Leblon,2015-02-08 07:16:49,2015-02-08 08:08:49,0:52


In [111]:
sorted_near_initial_points = near_points_to_origin('A27629', inverted_initial_point)
sorted_near_initial_points

,timestamp,latitude,longitude,speed
0,08-02-2015 19:23:27,-22.988079,-43.228588,0


In [36]:
#llamada a la funcion all_travel_table para el id = A27604
resulta = all_travel_table('438', inverted_initial_point, 'Leblon', inverted_final_point, 'Vila_Isabel', 'A27604')
resulta

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,438,A27604,Vila_Isabel_to_Leblon,2015-02-08 05:51:20,2015-02-08 14:20:03,8:28
1,438,A27604,Leblon_to_Vila_Isabel,2015-02-08 14:20:03,2015-02-08 15:31:33,1:11
2,438,A27604,Vila_Isabel_to_Leblon,2015-02-08 16:22:44,2015-02-08 17:17:14,0:54
3,438,A27604,Leblon_to_Vila_Isabel,2015-02-08 17:17:14,2015-02-08 18:27:14,1:10


In [1]:
#de aqui para abajo solo estoy probando

In [ ]:
sorted_near_initial_points_A27611 = near_points_to_origin ("A27611", (-43.228299999999997, -22.988199999999999))
sorted_near_initial_points_A27611

In [ ]:
sorted_near_final_points_A27611 = near_points_to_origin ("A27514", (-43.251199999999997, -22.9163))
sorted_near_final_points_A27611

In [ ]:
#pintando los puntos más cerca del punto inicial
fmap=folium.Map(location=[-22.950635, -43.210436], zoom_start=12)

for row in sorted_near_initial_points_A27611.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='violet', num_sides=4, radius=6)

embed_map(fmap_A27611)

In [ ]:
#pintando los puntos más cerca del punto final (Vila Isabel)
for row in sorted_near_final_points_A27611.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap.polygon_marker( latlon, fill_color='red', num_sides=4, radius=6)

embed_map(fmap_A27611)

In [ ]:
#Pintando Circulos de radio de 1km alrededor de los puntos incial y final 
fmap.simple_marker(final_point, popup='Parada de Vila Isabel')
fmap.simple_marker(initial_point, popup='Parada Leblon')

#Punto inicial
fmap.circle_marker(location=final_point, radius=1000,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap.circle_marker(location=initial_point, radius=1000,line_color='blue',
                 fill_color='#3186cc', fill_opacity=0.2)

embed_m